# EDA - Weather Forecast

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

- Aseguramos al hacer la importación que las fechas sean de tipo Datetime y los decimales en caso de tener comas se sustituya por puntos.

In [ ]:
df = pd.read_csv('../Data/raw/open_meteo/historico_provincias_espana.csv')

In [ ]:
df.head(5)

- Vemos la información general de nuestro dataset.

In [ ]:
df.info()

- Borramos la columna *date*

In [ ]:
df.drop(columns=['date'], inplace=True)

- Comprobamos los valores nulos.

In [ ]:
df.isna().sum()

- Previamente vemos que hay diferentes datos en un mismo día sin tener en cuenta las horas, por lo que realizamos una agrupación de los datos por día segun la media.

In [ ]:
df.shape

## Histograma

- Extraemos la lista de provincias

In [ ]:
provincias = df['provincia'].unique()
print(provincias)

In [ ]:
df_diario_madrid = df[df['provincia'] == 'Madrid']

In [ ]:
df_diario_madrid.sort_values(by='time', ascending=True, inplace=True)

### Temperatura media

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = df_diario_madrid['time'],
        y = df_diario_madrid['temperature_2m_mean'],
        mode = 'lines',
        name = "MADRID"
    )
)

fig.update_layout(
    title = f"Evolución temporal de temperatura media para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Temperatura Media (temperature_2m_mean)"
)

fig.show()

### Temperatura máxima

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = df_diario_madrid['time'],
        y = df_diario_madrid['temperature_2m_max'],
        mode = 'lines',
        name = "MADRID"
    )
)

fig.update_layout(
    title = f"Evolución temporal de temperatura máxima para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Temperatura Máxima (temperature_2m_max)"
)

fig.show()

### Temperatura mínima

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = df_diario_madrid['time'],
        y = df_diario_madrid['temperature_2m_min'],
        mode = 'lines',
        name = "MADRID"
    )
)

fig.update_layout(
    title = f"Evolución temporal de temperatura mínima para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Temperatura Mínima (temperature_2m_min)"
)

fig.show()

### Velocidad máxima *(km/h)*

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_diario_madrid['time'],
        y=df_diario_madrid['windspeed_10m_max'],
        name="MADRID",
        mode='lines'
    )
)

fig.update_layout(
    title = f"Evolución temporal de velocidad máxima del viento(km/h) para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Velocidad media (windspeed_10m_max)"
)

fig.show()

### Precipitaciones *(mL)*

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=df_diario_madrid['time'],
        y=df_diario_madrid['precipitation_sum'],
        name="MADRID"
    )
)

fig.update_layout(
    title = f"Evolución temporal de precipitaciones en milímetros para Madrid",
    xaxis_title = "Fecha",
    yaxis_title = "Precipitaciones (precipitation_sum)"
)

fig.show()

## Tendencia, estacionalidad y ruido

### Temperatura media

In [ ]:
decomposition_temp_mean = seasonal_decompose(df_diario_madrid['temperature_2m_mean'], model='additive', period=365)

In [ ]:
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(x=df_diario_madrid['time'], y=decomposition_temp_mean.trend, mode='lines', name='Tendencia'))
fig_trend.update_layout(title="Tendencia de la Temperatura Media", xaxis_title="Fecha", yaxis_title="Temperatura (°C)")
fig_trend.show()

### Temperatura máxima

### Temperatura mínima

### Velocidad máxima del viento *(km/h)*

### Precipitaciones máximas *(mL)*

## Detección de Outliers